In [8]:
import warnings
warnings.filterwarnings("ignore")

import sys, os
path_current = '/home/huzuntao/PycharmProjects/MPP_Powersystem/'
path_ = os.getcwd()
if path_current not in sys.path:
    sys.path.insert(1, '/home/huzuntao/PycharmProjects/MPP_Powersystem/')
elif path_ not in sys.path:
    sys.path.insert(1, path_)

from pypower_.pipsopf_solver import pipsopf_solver_gpu
from pypower__.pipsopf_solver import pipsopf_solver
import pandapower as pp

In [2]:
net = pp.create_empty_network()

# create buses
bus1 = pp.create_bus(net, vn_kv=110.)
bus2 = pp.create_bus(net, vn_kv=110.)
bus3 = pp.create_bus(net, vn_kv=110.)
bus4 = pp.create_bus(net, vn_kv=110.)
bus5 = pp.create_bus(net, vn_kv=110.)
bus6 = pp.create_bus(net, vn_kv=110.)

# create 110 kV lines
pp.create_line(net, bus4, bus5, length_km=90., std_type='149-AL1/24-ST1A 110.0')
pp.create_line(net, bus3, bus4, length_km=90., std_type='149-AL1/24-ST1A 110.0')
pp.create_line(net, bus2, bus3, length_km=90., std_type='149-AL1/24-ST1A 110.0')
pp.create_line(net, bus1, bus2, length_km=70., std_type='149-AL1/24-ST1A 110.0')
pp.create_line(net, bus6, bus3, length_km=70., std_type='149-AL1/24-ST1A 110.0')

# create loads
p_load_1 = 10
p_load_2 = 30
pp.create_load(net, bus2, p_mw=p_load_1, controllable=False)
pp.create_load(net, bus4, p_mw=p_load_2/2, controllable=False)
pp.create_load(net, bus5, p_mw=p_load_2/2, controllable=False)
pp.create_load(net, bus6, p_mw=p_load_2/2, controllable=False)
# create generators
eg = pp.create_ext_grid(net, bus1, min_p_mw=0, max_p_mw=1000, vm_pu=1.05)
g0 = pp.create_gen(net, bus3, p_mw=80, min_p_mw=0, max_p_mw=80, vm_pu=1.00, controllable=True)

costeg = pp.create_poly_cost(net, 0, 'ext_grid', cp1_eur_per_mw=20)
costgen1 = pp.create_poly_cost(net, 0, 'gen', cp1_eur_per_mw=10)
costgen2 = pp.create_poly_cost(net, 1, 'gen', cp1_eur_per_mw=10)

net.bus["min_vm_pu"] = 0.96
net.bus["max_vm_pu"] = 1.04
net.line["max_loading_percent"] = 100

In [3]:
om, ppopt, raw = pp.runopp(net, delta=1e-16, RETURN_RAW_DER=1)

In [4]:
results, success, raw = pipsopf_solver(om, ppopt)

In [9]:

results, success, raw = pipsopf_solver_gpu(om, ppopt)

A_: None; eyex shape: (16, 16); nx : 16
max it: 150; i: 1
max it: 150; i: 2
max it: 150; i: 3
max it: 150; i: 4
max it: 150; i: 5
max it: 150; i: 6
max it: 150; i: 7
max it: 150; i: 8
max it: 150; i: 9
max it: 150; i: 10
max it: 150; i: 11
max it: 150; i: 12
max it: 150; i: 13
max it: 150; i: 14
max it: 150; i: 15
max it: 150; i: 16
max it: 150; i: 17
max it: 150; i: 18
max it: 150; i: 19
max it: 150; i: 20
max it: 150; i: 21
max it: 150; i: 22
max it: 150; i: 23
max it: 150; i: 24
max it: 150; i: 25
max it: 150; i: 26


## pipscopf.py

In [35]:
from numpy import flatnonzero as find, ones, zeros, Inf, pi, exp, conj, r_
from pypower__.idx_brch import F_BUS, T_BUS, RATE_A, PF, QF, PT, QT, MU_SF, MU_ST
from pypower__.idx_bus import BUS_TYPE, REF, VM, VA, MU_VMAX, MU_VMIN, LAM_P, LAM_Q
from pypower__.idx_cost import MODEL, PW_LINEAR, NCOST
from pypower__.idx_gen import GEN_BUS, PG, QG, VG, MU_PMAX, MU_PMIN, MU_QMAX, MU_QMIN
from pypower__.makeYbus import makeYbus
from pypower__.opf_consfcn import opf_consfcn
from pypower__.opf_costfcn import opf_costfcn


from pypower__.opf_hessfcn import opf_hessfcn
from pypower__.pips import pips

# pip.py
from numpy import array, Inf, any, isnan, ones, r_, finfo, \
    zeros, dot, absolute, log, flatnonzero as find
from numpy.linalg import norm

from scipy.sparse import vstack, hstack, eye, csr_matrix as sparse
from scipy.sparse.linalg import spsolve
EPS = finfo(float).eps


In [36]:
x0_init=None
out_opt=None

if out_opt is None:
    out_opt = {}

## options
verbose = ppopt['VERBOSE']
feastol = ppopt['PDIPM_FEASTOL']
gradtol = ppopt['PDIPM_GRADTOL']
comptol = ppopt['PDIPM_COMPTOL']
costtol = ppopt['PDIPM_COSTTOL']
max_it  = ppopt['PDIPM_MAX_IT']
max_red = ppopt['SCPDIPM_RED_IT']
init = ppopt['INIT']
step_control = (ppopt['OPF_ALG'] == 565)  ## OPF_ALG == 565, PIPS-sc
if feastol == 0:
    feastol = ppopt['OPF_VIOLATION']
opt = {  'feastol': feastol,
         'gradtol': gradtol,
         'comptol': comptol,
         'costtol': costtol,
         'max_it': max_it,
         'max_red': max_red,
         'step_control': step_control,
         'cost_mult': 1e-0,
         'verbose': verbose  }

## unpack data
ppc = om.get_ppc()
baseMVA, bus, gen, branch, gencost = \
    ppc["baseMVA"], ppc["bus"], ppc["gen"], ppc["branch"], ppc["gencost"]
vv, _, nn, _ = om.get_idx()

## problem dimensions
nb = bus.shape[0]          ## number of buses
nl = branch.shape[0]       ## number of branches
ny = om.getN('var', 'y')   ## number of piece-wise linear costs

## linear constraints
A, l, u = om.linear_constraints()

## bounds on optimization vars
x0, xmin, xmax = om.getv()

## build admittance matrices
Ybus, Yf, Yt = makeYbus(baseMVA, bus, branch)

## try to select an interior initial point if init is not available from a previous powerflow
if init != "pf":
    ll, uu = xmin.copy(), xmax.copy()
    ll[xmin == -Inf] = -1e10   ## replace Inf with numerical proxies
    uu[xmax ==  Inf] =  1e10
    x0 = (ll + uu) / 2
    Varefs = bus[bus[:, BUS_TYPE] == REF, VA] * (pi / 180)
    ## angles set to first reference angle
    x0[vv["i1"]["Va"]:vv["iN"]["Va"]] = Varefs[0]
    if ny > 0:
        pass
#         ipwl = find(gencost[:, MODEL] == PW_LINEAR)
# #         PQ = r_[gen[:, PMAX], gen[:, QMAX]]
# #         c = totcost(gencost[ipwl, :], PQ[ipwl])
#         c = gencost.flatten('F')[sub2ind(gencost.shape, ipwl, NCOST+2*gencost[ipwl, NCOST])]    ## largest y-value in CCV data
#         x0[vv["i1"]["y"]:vv["iN"]["y"]] = max(c) + 0.1 * abs(max(c))
# #        x0[vv["i1"]["y"]:vv["iN"]["y"]] = c + 0.1 * abs(c)


## find branches with flow limits
il = find((branch[:, RATE_A] != 0) & (branch[:, RATE_A] < 1e10))
nl2 = len(il)           ## number of constrained lines

##-----  run opf  -----
f_fcn = lambda x, return_hessian=False: opf_costfcn(x, om, return_hessian)
gh_fcn = lambda x: opf_consfcn(x, om, Ybus, Yf[il, :], Yt[il,:], ppopt, il)
hess_fcn = lambda x, lmbda, cost_mult: opf_hessfcn(x, lmbda, om, Ybus, Yf[il, :], Yt[il, :], ppopt, il, cost_mult)



In [37]:
solution = pips(f_fcn, x0, A, l, u, xmin, xmax, gh_fcn, hess_fcn, opt)
x, f, info, lmbda, output = solution["x"], solution["f"], \
        solution["eflag"], solution["lmbda"], solution["output"]

success = (info > 0)

## pip.py


In [89]:
if isinstance(f_fcn, dict):  ## problem dict
    p = f_fcn
    f_fcn = p['f_fcn']
    x0 = p['x0']
    if 'opt' in p: opt = p['opt']
    if 'hess_fcn' in p: hess_fcn = p['hess_fcn']
    if 'gh_fcn' in p: gh_fcn = p['gh_fcn']
    if 'xmax' in p: xmax = p['xmax']
    if 'xmin' in p: xmin = p['xmin']
    if 'u' in p: u = p['u']
    if 'l' in p: l = p['l']
    if 'A' in p: A = p['A']

nx = x0.shape[0]                        # number of variables
nA = A.shape[0] if A is not None else 0 # number of original linear constr

# default argument values
if l is None or len(l) == 0: l = -Inf * ones(nA)
if u is None or len(u) == 0: u =  Inf * ones(nA)
if xmin is None or len(xmin) == 0: xmin = -Inf * ones(x0.shape[0])
if xmax is None or len(xmax) == 0: xmax =  Inf * ones(x0.shape[0])
if gh_fcn is None:
    nonlinear = False
    gn = array([])
    hn = array([])
else:
    nonlinear = True

if opt is None: opt = {}
# options
if "feastol" not in opt:
    opt["feastol"] = 1e-06
if "gradtol" not in opt:
    opt["gradtol"] = 1e-06
if "comptol" not in opt:
    opt["comptol"] = 1e-06
if "costtol" not in opt:
    opt["costtol"] = 1e-06
if "max_it" not in opt:
    opt["max_it"] = 150
if "max_red" not in opt:
    opt["max_red"] = 20
if "step_control" not in opt:
    opt["step_control"] = False
if "cost_mult" not in opt:
    opt["cost_mult"] = 1
if "verbose" not in opt:
    opt["verbose"] = 0

# initialize history
hist = []

# constants
xi = 0.99995
sigma = 0.1
z0 = 1
alpha_min = 1e-8
rho_min = 0.95
rho_max = 1.05
mu_threshold = 1e-5

# initialize
i = 0                       # iteration counter
converged = False           # flag
eflag = False               # exit flag

# add var limits to linear constraints
eyex = eye(nx, nx, format="csr")
AA = eyex if A is None else vstack([eyex, A], "csr")
ll = r_[xmin, l]
uu = r_[xmax, u]

# split up linear constraints
ieq = find( absolute(uu - ll) <= EPS )
igt = find( (uu >=  1e10) & (ll > -1e10) )
ilt = find( (ll <= -1e10) & (uu <  1e10) )
ibx = find( (absolute(uu - ll) > EPS) & (uu < 1e10) & (ll > -1e10) )
# zero-sized sparse matrices unsupported
Ae = AA[ieq, :] if len(ieq) else None
if len(ilt) or len(igt) or len(ibx):
    idxs = [(1, ilt), (-1, igt), (1, ibx), (-1, ibx)]
    Ai = vstack([sig * AA[idx, :] for sig, idx in idxs if len(idx)], 'csr')
else:
    Ai = None
be = uu[ieq]
bi = r_[uu[ilt], -ll[igt], uu[ibx], -ll[ibx]]

# evaluate cost f(x0) and constraints g(x0), h(x0)
x = x0
f, df = f_fcn(x)                 # cost
f = f * opt["cost_mult"]
df = df * opt["cost_mult"]
if 1:
    hn, gn, dhn, dgn = gh_fcn(x)        # nonlinear constraints
    h = hn if Ai is None else r_[hn.reshape(len(hn),), Ai * x - bi] # inequality constraints
    g = gn if Ae is None else r_[gn, Ae * x - be] # equality constraints
    if (dhn is None) and (Ai is None):
        dh = None
    elif dhn is None:
        dh = Ai.T
    elif Ai is None:
        dh = dhn
    else:
        dh = hstack([dhn, Ai.T])

    if (dgn is None) and (Ae is None):
        dg = None
    elif dgn is None:
        dg = Ae.T
    elif Ae is None:
        dg = dgn
    else:
        dg = hstack([dgn, Ae.T])
else:
    h = -bi if Ai is None else Ai * x - bi        # inequality constraints
    g = -be if Ae is None else Ae * x - be        # equality constraints
    dh = None if Ai is None else Ai.T     # 1st derivative of inequalities
    dg = None if Ae is None else Ae.T     # 1st derivative of equalities

# some dimensions
neq = g.shape[0]           # number of equality constraints
niq = h.shape[0]           # number of inequality constraints
neqnln = gn.shape[0]       # number of nonlinear equality constraints
niqnln = hn.shape[0]       # number of nonlinear inequality constraints
nlt = len(ilt)             # number of upper bounded linear inequalities
ngt = len(igt)             # number of lower bounded linear inequalities
nbx = len(ibx)             # number of doubly bounded linear inequalities

# initialize gamma, lam, mu, z, e
gamma = 1                  # barrier coefficient
lam = zeros(neq)
z = z0 * ones(niq)
mu = z0 * ones(niq)
k = find(h < -z0)
z[k] = -h[k]
k = find((gamma / z) > z0)
mu[k] = gamma / z[k]
e = ones(niq)

# check tolerance
f0 = f
if opt["step_control"]:
    L = f + dot(lam, g) + dot(mu, h + z) - gamma * sum(log(z))

Lx = df.copy()
Lx = Lx + dg * lam if dg is not None else Lx
Lx = Lx + dh * mu  if dh is not None else Lx

maxh = zeros(1) if len(h) == 0 else max(h)

gnorm = norm(g, Inf) if len(g) else 0.0
lam_norm = norm(lam, Inf) if len(lam) else 0.0
mu_norm = norm(mu, Inf) if len(mu) else 0.0
znorm = norm(z, Inf) if len(z) else 0.0
feascond = \
    max([gnorm, maxh]) / (1 + max([norm(x, Inf), znorm]))
gradcond = \
    norm(Lx, Inf) / (1 + max([lam_norm, mu_norm]))
compcond = dot(z, mu) / (1 + norm(x, Inf))
costcond = absolute(f - f0) / (1 + absolute(f0))

# save history
hist.append({'feascond': feascond, 'gradcond': gradcond,
    'compcond': compcond, 'costcond': costcond, 'gamma': gamma,
    'stepsize': 0, 'obj': f / opt["cost_mult"], 'alphap': 0, 'alphad': 0})

if opt["verbose"]: # pragma: no cover
    s = '-sc' if opt["step_control"] else ''
    v = {'Name': 'PIPS',
           'Version': '1.0',
           'Release':  '',
           'Date': '07-Feb-2011'}
    print('Python Interior Point Solver - PIPS%s, Version %s, %s' %
                (s, v['Version'], v['Date']))
    if opt['verbose'] > 1:
        print(" it    objective   step size   feascond     gradcond     "
              "compcond     costcond  ")
        print("----  ------------ --------- ------------ ------------ "
              "------------ ------------")
        print("%3d  %12.8g %10s %12g %12g %12g %12g" %
            (i, (f / opt["cost_mult"]), "",
             feascond, gradcond, compcond, costcond))

if feascond < opt["feastol"] and gradcond < opt["gradtol"] and \
    compcond < opt["comptol"] and costcond < opt["costtol"]:
    converged = True
    if opt["verbose"]:
        print("Converged!")

In [90]:
print(dh)

  (0, 3)	398.4826377065583
  (0, 8)	-398.48263770655285
  (1, 2)	379.507274006251
  (1, 3)	-398.4826377065592
  (1, 7)	-379.507274006251
  (1, 8)	398.48263770655194
  (2, 1)	379.507274006251
  (2, 2)	-379.507274006251
  (2, 4)	-379.5072740062469
  (2, 6)	-379.507274006251
  (2, 7)	379.507274006251
  (2, 9)	379.5072740062469
  (3, 0)	379.507274006251
  (3, 1)	-379.507274006251
  (3, 5)	-379.507274006251
  (3, 6)	379.507274006251
  (4, 0)	-379.507274006251
  (4, 5)	379.507274006251
  (5, 4)	379.5072740062469
  (5, 9)	-379.5072740062469
  (6, 3)	13643.858021102305
  (6, 8)	15325.320030992641
  (7, 2)	-797.5707992417994
  (7, 3)	-13681.114547475901
  (7, 7)	802.0514553740358
  :	:
  (9, 1)	802.0514553740358
  (9, 5)	802.0514553740358
  (9, 6)	-797.5707992417994
  (10, 0)	802.0514553740358
  (10, 5)	-797.5707992417994
  (11, 4)	-799.3409349977359
  (11, 9)	802.0514553740269
  (7, 10)	1.0
  (8, 11)	1.0
  (9, 12)	1.0
  (10, 13)	1.0
  (11, 14)	1.0
  (12, 15)	1.0
  (13, 16)	1.0
  (14, 17)	1.0
 

In [88]:
print(Ai)

  (0, 7)	1.0
  (1, 8)	1.0
  (2, 9)	1.0
  (3, 10)	1.0
  (4, 11)	1.0
  (5, 12)	1.0
  (6, 13)	1.0
  (7, 14)	1.0
  (8, 15)	1.0
  (9, 7)	-1.0
  (10, 8)	-1.0
  (11, 9)	-1.0
  (12, 10)	-1.0
  (13, 11)	-1.0
  (14, 12)	-1.0
  (15, 13)	-1.0
  (16, 14)	-1.0
  (17, 15)	-1.0


### Iteration loop

In [91]:
# do Newton iterations
while (not converged) and (i < 1):
    # update iteration counter
    i += 1
    print(f"max it: {opt['max_it']}; i: {i}")
    # compute update step
    lmbda = {"eqnonlin": lam[range(neqnln)],
             "ineqnonlin": mu[range(niqnln)]}
    if nonlinear:
        if hess_fcn is None:
            print("pips: Hessian evaluation via finite differences "
                  "not yet implemented.\nPlease provide "
                  "your own hessian evaluation function.")
        Lxx = hess_fcn(x, lmbda, opt["cost_mult"])
    else:
        _, _, d2f = f_fcn(x, True)      # cost
        Lxx = d2f * opt["cost_mult"]
    rz = range(len(z))
    zinvdiag = sparse((1.0 / z, (rz, rz))) if len(z) else None
    rmu = range(len(mu))
    mudiag = sparse((mu, (rmu, rmu))) if len(mu) else None
    dh_zinv = None if dh is None else dh * zinvdiag
    M = Lxx if dh is None else Lxx + dh_zinv * mudiag * dh.T
    N = Lx if dh is None else Lx + dh_zinv * (mudiag * h + gamma * e)

    Ab = sparse(M) if dg is None else vstack([
        hstack([M, dg]),
        hstack([dg.T, sparse((neq, neq))])
    ])
    bb = r_[-N, -g]

    dxdlam = spsolve(Ab.tocsr(), bb)

    if any(isnan(dxdlam)):
        if opt["verbose"]:
            print('\nNumerically Failed\n')
        eflag = -1
        break

    dx = dxdlam[:nx]
    dlam = dxdlam[nx:nx + neq]
    dz = -h - z if dh is None else -h - z - dh.T * dx
    dmu = -mu if dh is None else -mu + zinvdiag * (gamma * e - mudiag * dz)

    # do the update
    k = find(dz < 0.0)
    alphap = min([xi * min(z[k] / -dz[k]), 1]) if len(k) else 1.0
    k = find(dmu < 0.0)
    alphad = min([xi * min(mu[k] / -dmu[k]), 1]) if len(k) else 1.0
    x = x + alphap * dx
    z = z + alphap * dz
    lam = lam + alphad * dlam
    mu = mu + alphad * dmu
    if niq > 0:
        gamma = sigma * dot(z, mu) / niq

    # evaluate cost, constraints, derivatives
    f, df = f_fcn(x)             # cost
    f = f * opt["cost_mult"]
    df = df * opt["cost_mult"]
    if nonlinear:
        hn, gn, dhn, dgn = gh_fcn(x)                   # nln constraints
#            g = gn if Ai is None else r_[gn, Ai * x - bi] # ieq constraints
#            h = hn if Ae is None else r_[hn, Ae * x - be] # eq constraints
        h = hn if Ai is None else r_[hn.reshape(len(hn),), Ai * x - bi] # ieq constr
        g = gn if Ae is None else r_[gn, Ae * x - be]  # eq constr

        if (dhn is None) and (Ai is None):
            dh = None
        elif dhn is None:
            dh = Ai.T
        elif Ai is None:
            dh = dhn
        else:
            dh = hstack([dhn, Ai.T])

        if (dgn is None) and (Ae is None):
            dg = None
        elif dgn is None:
            dg = Ae.T
        elif Ae is None:
            dg = dgn
        else:
            dg = hstack([dgn, Ae.T])
    else:
        h = -bi if Ai is None else Ai * x - bi    # inequality constraints
        g = -be if Ae is None else Ae * x - be    # equality constraints
        # 1st derivatives are constant, still dh = Ai.T, dg = Ae.T

    Lx = df
    Lx = Lx + dg * lam if dg is not None else Lx
    Lx = Lx + dh * mu  if dh is not None else Lx

    if len(h) == 0:
        maxh = zeros(1)
    else:
        maxh = max(h)

    gnorm = norm(g, Inf) if len(g) else 0.0
    lam_norm = norm(lam, Inf) if len(lam) else 0.0
    mu_norm = norm(mu, Inf) if len(mu) else 0.0
    znorm = norm(z, Inf) if len(z) else 0.0
    feascond = \
        max([gnorm, maxh]) / (1 + max([norm(x, Inf), znorm]))
    gradcond = \
        norm(Lx, Inf) / (1 + max([lam_norm, mu_norm]))
    compcond = dot(z, mu) / (1 + norm(x, Inf))
    costcond = float(absolute(f - f0) / (1 + absolute(f0)))

    hist.append({'feascond': feascond, 'gradcond': gradcond,
        'compcond': compcond, 'costcond': costcond, 'gamma': gamma,
        'stepsize': norm(dx), 'obj': f / opt["cost_mult"],
        'alphap': alphap, 'alphad': alphad})

    if opt["verbose"] > 1:
        print("%3d  %12.8g %10.5g %12g %12g %12g %12g" %
            (i, (f / opt["cost_mult"]), norm(dx), feascond, gradcond,
             compcond, costcond))

    if feascond < opt["feastol"] and gradcond < opt["gradtol"] and \
        compcond < opt["comptol"] and costcond < opt["costtol"]:
        converged = True
        if opt["verbose"]:
            print("Converged!")
    else:
        if any(isnan(x)) or (alphap < alpha_min) or \
            (alphad < alpha_min) or (gamma < EPS) or (gamma > 1.0 / EPS):
            if opt["verbose"]:
                print("Numerically failed.")
            eflag = -1
            break
        f0 = f

        if opt["step_control"]:
            L = f + dot(lam, g) + dot(mu, (h + z)) - gamma * sum(log(z))


max it: 150; i: 1


In [65]:
print(f"Lxx: {Lxx.toarray()}")


Lxx: [[ 6.08292972e+05 -6.08292972e+05  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  7.27595761e-12  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [-6.08292972e+05  9.58116109e+05 -3.49823137e+05  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00 -7.27595761e-12
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00 -3.49823137e+05  1.27897291e+06 -3.49823137e+05
   0.00000000e+00 -5.79326640e+05  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00 -3.49823137e+05  6.99646273e+05
  -3.49823137e+05  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  

In [66]:
print(f"dxdlam:{dxdlam}")

dxdlam:[ 0.00000000e+00 -1.08418363e-02  9.02859755e-03 -8.83492868e-02
 -1.36005826e-01 -2.77964824e-02  0.00000000e+00  2.23184917e-02
 -8.75397820e-03 -3.95947024e-02 -5.71969480e-02 -2.29541905e-02
 -4.91340899e+02  6.38460652e+00  6.87341542e+00 -2.01218903e+01
  1.80346364e+01  1.60868888e+01  1.03192303e+01  1.29531655e+02
  1.89162197e+02  7.01420314e+01  1.37468308e-08 -4.26652326e+01
 -4.02437806e-08 -1.67696619e+01 -2.22526668e+01 -4.45644602e+00
 -7.46005065e-12 -1.15387682e+03]


In [67]:
print(f"Ab:{Ab}\nbb:{bb}")

Ab:  (0, 7)	91.44355891476664
  (0, 6)	-95.61317967496848
  (0, 1)	-608334.4586067942
  (0, 0)	608334.4586067941
  (1, 8)	75.72801186223592
  (1, 6)	95.61317967497234
  (1, 7)	-167.17157077700642
  (1, 2)	-349859.0693443677
  (1, 1)	958193.5279511621
  (1, 0)	-608334.4586067942
  (2, 8)	-227.25946669852146
  (2, 9)	75.72801186223592
  (2, 7)	75.72801186223592
  (2, 11)	75.80344297404963
  (2, 5)	-579362.5687947819
  (2, 3)	-349859.0693443677
  (2, 2)	1279080.7074835175
  (2, 1)	-349859.0693443677
  (3, 9)	-151.45602372447183
  (3, 10)	75.72801186223592
  (3, 8)	75.72801186223592
  (3, 4)	-349859.0693443677
  (3, 3)	699718.1386887354
  (3, 2)	-349859.0693443677
  (4, 9)	75.72801186223592
  :	:
  (21, 8)	-162.99668368338897
  (23, 8)	-267.9269657338639
  (24, 8)	872.0160557848681
  (25, 8)	-267.9269657338639
  (27, 8)	-344.47752737211067
  (18, 9)	-126.77519842041363
  (19, 9)	253.55039684082726
  (20, 9)	-126.77519842041363
  (24, 9)	-267.9269657338639
  (25, 9)	529.8668412681491
  (26,

In [68]:
Ab.shape

(30, 30)

In [74]:
print(Lx + dh_zinv * (mudiag * h + gamma * e))

[-4.26075931e+03  4.46502602e+03 -2.92419566e+02  3.00365780e+02
 -3.75242041e+02  1.63029113e+02  2.67629078e+07 -2.67540988e+07
  9.56307214e+03  6.96658792e+03  3.44657682e+03  2.16737614e+03
  7.97093165e+00  5.26530355e+00  1.35137482e-08 -3.95614323e-08]


In [73]:
(dh_zinv * (mudiag * h + gamma * e)).shape

(16,)

In [76]:
print(Lx)

[-3.15431743e+02  5.22738110e+02 -2.96708652e+02  3.01118514e+02
 -3.74963740e+02  1.63247510e+02  1.49305720e+04 -1.56026999e+04
 -7.15759503e+02 -9.33998102e+02 -5.03256224e+02 -2.21897926e+02
  9.90297178e+00  4.95148589e+00  9.69417421e-17  8.37681472e-17]


In [78]:
print(dh_zinv * (mudiag * h + gamma * e))

[-3.94532757e+03  3.94228791e+03  4.28908589e+00 -7.52733278e-01
 -2.78300925e-01 -2.18396961e-01  2.67479772e+07 -2.67384961e+07
  1.02788316e+04  7.90058602e+03  3.94983304e+03  2.38927407e+03
 -1.93204013e+00  3.13817659e-01  1.35137481e-08 -3.95614324e-08]


In [81]:
print(dg.toarray())

[[ 371.48023149 -367.75209887    0.            0.            0.
     0.         -170.95202974  178.83107291    0.            0.
     0.            0.            1.            0.        ]
 [-371.48023149  636.64061076 -273.90603262    0.            0.
     0.          170.95202974 -312.55033813  123.11522658    0.
     0.            0.            0.            0.        ]
 [   0.         -268.8885119   878.8698316  -242.56827256    0.
  -327.87159981    0.          133.71926522 -364.71706536  144.63921783
     0.          169.92555861    0.            0.        ]
 [   0.            0.         -265.65906737  490.70671829 -237.3681742
     0.            0.            0.           95.83908448 -248.12202211
   126.24471832    0.            0.            0.        ]
 [   0.            0.            0.         -248.13844573  237.3681742
     0.            0.            0.            0.          103.48280428
  -126.24471832    0.            0.            0.        ]
 [   0.            0.      

In [92]:
print(dh)

  (0, 3)	3719.8563715503165
  (0, 8)	2905.451877035773
  (1, 2)	-3076.9104309543004
  (1, 3)	-3719.856371550317
  (1, 7)	-3845.7560653242244
  (1, 8)	-2905.451877035773
  (2, 1)	16289.110371959681
  (2, 2)	3076.9104309543004
  (2, 4)	9791.465789094711
  (2, 6)	15569.22772847432
  (2, 7)	3845.7560653242244
  (2, 9)	10526.486946351524
  (3, 0)	7773.714316406145
  (3, 1)	-16289.110371959683
  (3, 5)	7086.01461683815
  (3, 6)	-15569.227728474321
  (4, 0)	-7773.714316406145
  (4, 5)	-7086.01461683815
  (5, 4)	-9791.465789094713
  (5, 9)	-10526.486946351522
  (6, 3)	7309.569641931692
  (6, 8)	8982.764738383588
  (7, 2)	4553.446000678547
  (7, 3)	-7286.761694761755
  (7, 7)	6202.861790876377
  :	:
  (9, 1)	-3702.2325089789338
  (9, 5)	3961.8136523955686
  (9, 6)	-5311.514904418306
  (10, 0)	-2068.1566495466054
  (10, 5)	-3611.998832396194
  (11, 4)	-4650.312866919104
  (11, 9)	-3057.8214328422127
  (7, 10)	1.0
  (8, 11)	1.0
  (9, 12)	1.0
  (10, 13)	1.0
  (11, 14)	1.0
  (12, 15)	1.0
  (13, 16)

In [30]:
x_0, _, _ = om.getv()
x_0

array([ 0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  1.05,  1.  ,  1.  ,
        1.  ,  1.  ,  1.  ,  0.  , 80.  ,  0.  ,  0.  ])